## Import Libiray

In [1]:
import pandas as pd
import numpy as np
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import winsound

## Define Parameters

In [2]:
ts_data_add = 'CNN_Pred_QPL/'
origin_data_save = 'TS_data/'
model_add = 'lstm_model_save_QPL/'
# header = ['Open','High','Low','Close','QPL1','QPL2','QPL3','QPL4','QPL5','QPL6','QPL7','QPL8','QPL9','QPL10',
#                 'QPL11','QPL12','QPL13','QPL14','QPL15','QPL16','QPL17','QPL18','QPL19','QPL20','QPL21','QPL-1','QPL-2','QPL-3','QPL-4',
#                  'QPL-5','QPL-6','QPL-7','QPL-8','QPL-9','QPL-10','QPL-11','QPL-12','QPL-13','QPL-14','QPL-15','QPL-16','QPL-17','QPL-18','QPL-19',
#                  'QPL-20','QPL-21']

window_size = 4      
n_pred_days = 1    

# Network Parameters
input_dim = 4        
output_dim = 2       
n_layers = 2         
n_hidden_size = 64   

# Hyper parameters
epochs = 1500        
train_prop = 0.8
learning_rate = 1e-2 #Learning rate

duration = 1000  # millisecond
freq = 440  # Hz

## DataLoader

In [3]:
product = 'NZDCHF'
ts_d = pd.read_csv(ts_data_add+product+'.csv')
# Change Header

#new_index = header
#ts_d.columns = new_index
ts_d = ts_d.dropna(axis=0,how='any')
# ts_d = ts_d.drop(['QPL1','QPL2','QPL3','QPL4','QPL5','QPL6','QPL7','QPL8','QPL9','QPL10',
#                 'QPL11','QPL12','QPL13','QPL14','QPL15','QPL16','QPL17','QPL18','QPL19','QPL20','QPL21','QPL-1','QPL-2','QPL-3','QPL-4',
#                  'QPL-5','QPL-6','QPL-7','QPL-8','QPL-9','QPL-10','QPL-11','QPL-12','QPL-13','QPL-14','QPL-15','QPL-16','QPL-17','QPL-18','QPL-19',
#                  'QPL-20','QPL-21'],axis=1)
# Minimize the first three days
n_samples = len(ts_d) - window_size+1
print(n_samples)
# data.shape: (len(ts_w)*4*4) 
data = np.zeros((len(ts_d), window_size, input_dim), dtype=float)

ts_d_n = ts_d.iloc[:,:].values
ts_d_n

1597


array([[0.7407983 , 0.74394715, 0.7373952 , 0.74027354],
       [0.73950666, 0.7426198 , 0.73612446, 0.73901874],
       [0.7375046 , 0.740611  , 0.7341334 , 0.7370463 ],
       ...,
       [0.6695665 , 0.67286366, 0.66667885, 0.6698684 ],
       [0.66669947, 0.67002326, 0.6638084 , 0.66702384],
       [0.6726449 , 0.6759166 , 0.6697346 , 0.6729222 ]])

In [4]:
# Generate Traing set
for i in range(0, n_samples):
    # Generate 4*4 temporary array
    tmp = np.zeros((window_size, input_dim))
    for j in range(i, i+window_size):
        # i=0, j=0,1,2,3 
        tmp[j-i]=ts_d_n[j]
    # Fill data array
    data[i] = tmp

n_train_samples = n_samples
train_data = data[0:n_train_samples]
# Initialize the training matrices
train_X = np.zeros((n_train_samples, window_size-n_pred_days, input_dim), dtype=float)
train_Y = np.zeros((n_train_samples, output_dim), dtype=float)
# Split 4*4 data into 3*4 and 1*2
for i in range(n_train_samples):
    train_X[i] = train_data[i][0:window_size-1]
    for j in range(output_dim):
        train_Y[i][j] = train_data[i][window_size-1][j+1]

# Encapsulate the data into torch tensor
train_X = torch.tensor(train_X,dtype=torch.float32).cuda()
train_Y = torch.tensor(train_Y,dtype=torch.float32).cuda()


print('Finish Loading........')
print('training data:',n_train_samples)

Finish Loading........
training data: 1597


In [5]:
ts_d = pd.read_csv(origin_data_save+product+'1440_Data.csv')
# Change Header

#new_index = header
#ts_d.columns = new_index
ts_d = ts_d.dropna(axis=0,how='any')
ts_d = ts_d.drop(['Date','Year','Month','Day','Hour','Minute','Volume','RSI','MACD_M','MACD_S','STO_K','STO_D'],axis=1)
# Minimize the first three days
n_samples = len(ts_d) - window_size+1
print(n_samples)
# data.shape: (len(ts_w)*4*4) 
data = np.zeros((len(ts_d), window_size, input_dim), dtype=float)

ts_d_n = ts_d.iloc[:,:].values
ts_d_n

1997


array([[0.73895, 0.73997, 0.7384 , 0.7394 ],
       [0.73935, 0.74258, 0.73573, 0.73686],
       [0.73685, 0.73905, 0.73577, 0.73701],
       ...,
       [0.58383, 0.59034, 0.58383, 0.58957],
       [0.58957, 0.59069, 0.58448, 0.585  ],
       [0.585  , 0.59362, 0.58466, 0.59311]])

In [6]:
# Generate Test set
for i in range(0, n_samples):
    # Generate 4*4 temporary array
    tmp = np.zeros((window_size, input_dim))
    for j in range(i, i+window_size):
        # i=0, j=0,1,2,3 
        tmp[j-i]=ts_d_n[j]
    # Fill data array
    data[i] = tmp
n_train_samples = math.ceil(train_prop*n_samples)
n_test_samples = n_samples-n_train_samples

test_data = data[n_train_samples: n_samples]

test_X = np.zeros((n_test_samples, window_size-n_pred_days, input_dim), dtype=float)
test_Y = np.zeros((n_test_samples, output_dim), dtype=float)

for i in range(n_test_samples):
    test_X[i] = test_data[i][0:window_size-1]
    for j in range(output_dim):
        test_Y[i][j] = test_data[i][window_size-1][j+1]


test_X = torch.tensor(test_X,dtype=torch.float32).cuda()
test_Y = torch.tensor(test_Y,dtype=torch.float32).cuda()

print('Finish Loading........')
print('test_data:',n_test_samples)
test_X

Finish Loading........
test_data: 399


tensor([[[0.6732, 0.6802, 0.6719, 0.6793],
         [0.6782, 0.6791, 0.6781, 0.6788],
         [0.6788, 0.6819, 0.6768, 0.6776]],

        [[0.6782, 0.6791, 0.6781, 0.6788],
         [0.6788, 0.6819, 0.6768, 0.6776],
         [0.6776, 0.6807, 0.6762, 0.6787]],

        [[0.6788, 0.6819, 0.6768, 0.6776],
         [0.6776, 0.6807, 0.6762, 0.6787],
         [0.6787, 0.6871, 0.6786, 0.6853]],

        ...,

        [[0.5908, 0.5918, 0.5806, 0.5825],
         [0.5817, 0.5824, 0.5799, 0.5801],
         [0.5801, 0.5846, 0.5786, 0.5838]],

        [[0.5817, 0.5824, 0.5799, 0.5801],
         [0.5801, 0.5846, 0.5786, 0.5838],
         [0.5838, 0.5903, 0.5838, 0.5896]],

        [[0.5801, 0.5846, 0.5786, 0.5838],
         [0.5838, 0.5903, 0.5838, 0.5896],
         [0.5896, 0.5907, 0.5845, 0.5850]]], device='cuda:0')

## Define LSTM Network

In [7]:
class LSTM(nn.Module): # extend nn.Module
    # Define the LSTM network
    def __init__(self,input_size,hidden_size, output_size=output_dim,num_layers=2):
        super(LSTM,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        # LSTM layer
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers)
        self.fc = nn.Linear(hidden_size*(window_size-n_pred_days),output_size)
        self.batch_first = True

    def forward(self,x): # x: input
        # Only get the result， no hidden state
        lstm_out, _ = self.lstm(x)
        # get the index of hidden state
        batch_n,win_s,hidden_s = lstm_out.shape
        # Flatten to 2D: s*b rows and h cols
        fc_in = lstm_out.view(batch_n, win_s*hidden_s)
        fc_out = self.fc(fc_in)
        output = fc_out.view(batch_n,2)
        return output


In [8]:
lstm = LSTM(input_size=input_dim, hidden_size=n_hidden_size).cuda()
print(lstm)

LSTM(
  (lstm): LSTM(4, 64, num_layers=2)
  (fc): Linear(in_features=192, out_features=2, bias=True)
)


In [9]:
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

In [10]:
record_period = 50

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

def train_model(train_X, save_model_name):
    losses = []
    losses2 = []
    print("Training Start. Epochs = ", epochs)
    for i in range(epochs):
        output = lstm(train_X)
        loss = loss_function(output, train_Y)
        loss2 = RMSELoss(output,train_Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        losses2.append(loss2.item())
        if i ==1499:
            winsound.Beep(freq, duration)
        if (i+1) % record_period == 0:
            print('Epoch: {}, Loss:{:.10f}'.format(i+1, loss.item()))
            
    torch.save(lstm, model_add+save_model_name)
    return loss.item(), losses,loss2.item()



In [11]:
def test_model(model_name, test_X):
    # Load the trained model
    lstm64 = torch.load(model_add+model_name)
    # Forward the test data
    test_output = lstm64.forward(test_X)
    # Calculate the loss
    loss = loss_function(test_output, test_Y)
    loss2 = RMSELoss(test_output,test_Y)

    return test_output, loss.item(),loss2.item()



In [ ]:
if __name__ == '__main__':
    model_name=product+'.pkl'
    train_MSE, losses,train_RMSE = train_model(train_X, model_name)

In [45]:
test_output, test_MSE, test_RMSE = test_model(model_name, test_X)

print("Train MSE ", train_MSE,";test MSE ", test_MSE,";Train RMSE",train_RMSE,"test RMSE",test_RMSE)


Train MSE  2.9288121368153952e-05 ;test MSE  3.11149560729973e-05 ;Train RMSE 0.005411850288510323 test RMSE 0.005578078329563141


In [13]:
test_output, _, _ = test_model(model_name, test_X)
test_output_plot = test_output.cpu().detach().numpy()
test_y_plot = test_Y.cpu().detach().numpy()

In [48]:
out = pd.DataFrame(test_output_plot)
out.to_csv('TS_data_Pred_QPL/'+product+'.csv',index=False)